In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250


cuda


In [19]:
with open('The_wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(len(chars))
vocab_size = len(chars)
print(chars)
    

80
['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—']


In [20]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
#print(data[:100])

In [21]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)
    

inputs:
tensor([[77, 12,  1,  3, 27, 73, 72,  1],
        [ 1, 71, 57, 57, 71,  1, 54, 57],
        [67, 71, 72,  1, 67, 74, 57, 70],
        [67, 66, 57, 12,  1, 26, 58, 72]], device='cuda:0')
targets:
tensor([[12,  1,  3, 27, 73, 72,  1, 34],
        [71, 57, 57, 71,  1, 54, 57, 72],
        [71, 72,  1, 67, 74, 57, 70, 10],
        [66, 57, 12,  1, 26, 58, 72, 57]], device='cuda:0')


In [22]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is',context,'target is',target)

when input is tensor([45]) target is tensor(60)
when input is tensor([45, 60]) target is tensor(57)
when input is tensor([45, 60, 57]) target is tensor(1)
when input is tensor([45, 60, 57,  1]) target is tensor(41)
when input is tensor([45, 60, 57,  1, 41]) target is tensor(70)
when input is tensor([45, 60, 57,  1, 41, 70]) target is tensor(67)
when input is tensor([45, 60, 57,  1, 41, 70, 67]) target is tensor(62)
when input is tensor([45, 60, 57,  1, 41, 70, 67, 62]) target is tensor(57)


In [23]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [24]:
ix = torch.randint(len(data)-block_size,(batch_size,))
print(ix)

tensor([103354,  65542, 120899,  60235])


In [25]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)        


dpM2O4llG?fEgci,SI*2PZR5!j-C(]N7Y"C6pL0pFlw :MKw.Zk7goT[G)NyG6Wzk[Hn—a&Cp51
*j'smgoN(*NT.0URbS
4Lib;IVK;qHIp3Z.N!G?;QEJ;Il;sl2Twb8)58EwFG*WCsYO;2?k-orJBLgID4zkqu86GTCG?J1(8lcq 6p
w-ySZf?r)N1ghF&9a0BM3p&Ee&k(*YDz02Fq";.RJz0v*s*2,NZjlsK5 -1oZrxm5eHs'kNyOLU-m7qvCfC)OL02ZH]Q5YDg"p;[a34Tnj
j
Px3p3jGn?1D..Q7GIS]9#C]8'E,rFK-WOT2w#JPHG?fm27z-Q558r*NRSJ
y)ZKY76Sl8r2D!J:
ew"y!7:
wL,z-dEm-)m 1vWOpZP)Vye!v—TG6drS&4lH254EIf957S
-UD1e'vYR.#&t(ap[*N)5—4H!PzaY&-KvAH4w.4l#xNy1—v*2u72grlud4EAIO1&ZcH-[sQD8:EC&Q
g,


In [26]:
#create a pytorch optimiser
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
 
for iter in range(max_iters):
    
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step:{iter},train_losses:{losses['train']:.3f},val:{losses['val']:.3f}")
        
    xb, yb = get_batch('train')
    
    #evaluate the loss
    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())    
     

step:0,train_losses:4.845,val:4.870
step:250,train_losses:4.775,val:4.807
step:500,train_losses:4.705,val:4.731
step:750,train_losses:4.643,val:4.674
4.50199031829834


In [27]:
context = torch.zeros((1,1),dtype=torch.long,device=device)
generate_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generate_chars)


6NWUAJZ.idvzt'Slq?r*FY"l0ru—Dk7]#irY—f(3Z7,OdHlK,6eSfrjq)(4)(*lDzk 1?fBYY0G*4Ts
C"oCZ.2Z.NWrwc]BZ.-0#go#J.gQFz#2
2'V9a9al—G?Y#F58nfgIjVw3hwh)wnY&f3py#sugL(-1yf—5AH#goZpSAmQD![I#s9R.n&CeSNpTRJPC7lM—eGUgod]pDzWzl3;sdetV'WzP3#oIfpZwm]GYs.26Qllsk7cha!U1
#(r-Mo[s0V8*N"C1eBH
w?;w#ojGH6NP[sArhf?p4HYm2JPMcsQbfBi"?ws--1pdFA?fx8rELDzE1QtL(R?poQ4B57cV3g#Z9ONu])l,"V;D1'sOE8OL((fe);4]M:2k]Qk8"NP#2,Bm1gW1(gxWrpdFnaEngu"gK(6)bSdo[xMe)N1Vh;s0F5*:LVyn,
QRogoZG?fg-K4lL!.8"V8JZ.Z"c:jmkcIl.Y;GYLwwcwFxG*rAkcAvEa02]k
